In [1]:
%load_ext autoreload
%autoreload 2
from PyHa.statistics import *
from PyHa.IsoAutio import *
from PyHa.visualizations import *
from PyHa.annotation_post_processing import *
import pandas as pd

In [9]:
path = "./mixed_bird/025_Mixed_Bird_Outputs_Train/"
manual_csv = "./mixed_bird_manual.csv"
device_name = "cpu"#"cuda:0"

In [10]:
from PyHa.tweetynet_package.tweetynet.Load_data_functions import compute_features, predictions_to_kaleidoscope
from PyHa.microfaune_package.microfaune import audio
import torch.optim as optim
import torch.nn as nn

# PREP

In [11]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class kaledoscope_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file,filepath, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.manual_df = pd.read_csv(csv_file)
        self.manual_df["FOLDER"] = self.manual_df["IN FILE"].apply(lambda x: filepath)
        self.audio_files = self.manual_df[["FOLDER","IN FILE"]].drop_duplicates().reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.manual_df)

    def __getitem__(self, idx):
        #TODO: FIX MAGIC NUMBERS
        normalized_sample_rate=44100

        audio_df = self.audio_files.iloc[idx]
        
        audio_dif = audio_df["FOLDER"]
        audio_file = audio_df["IN FILE"]
        try:
            SAMPLE_RATE, SIGNAL = audio.load_wav(audio_dif + audio_file)
        except BaseException as e:
            #print("Failed to load", audio_dif + audio_file)
            #print(e)
            return (-1,-1)
        
        try:
            if SAMPLE_RATE != normalized_sample_rate:
                rate_ratio = normalized_sample_rate / SAMPLE_RATE
                SIGNAL = scipy_signal.resample(
                    SIGNAL, int(len(SIGNAL) * rate_ratio))
                SAMPLE_RATE = normalized_sample_rate
        except:
            print("Failed to Downsample" + audio_file)
            return (-1,-1)
            
        if len(SIGNAL.shape) == 2:
            SIGNAL = SIGNAL.sum(axis=1) / 2
        # detection
        try:
            tweetynet_features = compute_features([SIGNAL])
        except:
            print("Failed to compute features" + audio_file)
            return (-1,-1)
        

            
        label_df = self.manual_df[(self.manual_df["FOLDER"] == audio_df["FOLDER"]) & (self.manual_df["IN FILE"] == audio_df["IN FILE"])]
        return (tweetynet_features, label_df, audio_dif, audio_file, SIGNAL, SAMPLE_RATE)
    
trainloader = kaledoscope_dataset(manual_csv, path)
trainloader

In [12]:
# init detector


# Training

In [13]:
# TODO: DEFINE TEST LOADER

In [14]:
def IOU_Loss(automated_df, manual_df):
    IoUMatrix = torch.tensor(clip_IoU(automated_df, manual_df));
    return torch.mean(IoUMatrix)

def convert_label_to_local_score(manual_df, size_of_local_sorce, start_time = 0, duration_of_clip = -1):
    if (manual_df.empty):
        return torch.tensor(np.ones(size_of_local_sorce)).float()
    if (duration_of_clip == -1):
        duration_of_clip = manual_df.iloc[0]["CLIP LENGTH"]
    seconds_per_index = duration_of_clip/size_of_local_sorce
    local_score = np.ones(size_of_local_sorce)
    for i in range(size_of_local_sorce):
        current_seconds = i * seconds_per_index + start_time
        annotations_at_time = manual_df[(manual_df["OFFSET"] <= current_seconds) & (manual_df["OFFSET"] +manual_df["DURATION"] >=  current_seconds)]
        if (not annotations_at_time.empty):
            local_score[i] = 0
    
    return torch.tensor(local_score).float()



manual_df  = pd.read_csv("ScreamingPiha_Manual_Labels.csv")
convert_label_to_local_score(manual_df[manual_df["IN FILE"] == "ScreamingPiha5.wav"], 500)

tensor([1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [15]:
%reload_ext autoreload
torch.autograd.set_detect_anomaly(True)
device = torch.device(device_name)
net_wrapper = TweetyNetModel(2, (1, 86, 86), 86, device)
net_wrapper.model = net_wrapper.model.to(device)


net = net_wrapper.model
for param in net.parameters():
    param.requires_grad = True
optimizer = optim.Adam(net_wrapper.model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    batch_loss = 0.0
    mini_batch_count = 0
    batch_count = 0
    file_count = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        try:
            inputs, labels_df, audio_dif, audio_file, SIGNAL, SAMPLE_RATE = data
        except:
            continue

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        net = net_wrapper.model
        for param in net.parameters():
            param.requires_grad = True
        
        batch_size=1
        window_size=2
        test_data_loader = DataLoader(inputs, batch_size=batch_size)
        predictions = pd.DataFrame()
        #net_wrapper.model.eval()
        local_score = torch.tensor([])
        tmp_local_score = []
        dataiter = iter(test_data_loader)
        _, label, uid = dataiter.next()
        time_bin = float(window_size)/label.shape[1]
        st_time = np.array([time_bin*n for n in range(label.shape[1])])
        count = []
        
        total_count = 0
        ones = 0
        zeros = 0

        
        for i, data in enumerate(test_data_loader):
                
            #Run Model
            sub_inputs, sub_labels, uids = data
            sub_inputs = sub_inputs.clone().detach().requires_grad_(True)
            sub_inputs, sub_labels = sub_inputs.to(device), sub_labels.to(device)
            output = net_wrapper.model.to(device)(sub_inputs, sub_inputs.shape[0], sub_inputs.shape[0])

            #Get labels for window
            bins = st_time + (int(uids[0].split("_")[0])*window_size)
            duration_of_window = bins[-1] -bins[0]
            #print(convert_label_to_local_score(labels_df, len([x for x in output[0, 1, :]])))
            label_for_output = convert_label_to_local_score(labels_df, 86, bins[0],duration_of_window)
            
            
            total_count += 86
            ones += sum(label_for_output)
            zeros += 86 - sum(label_for_output)
            
            
            
            #label_for_output = convert_label_to_local_score(labels_df, len([x for x in output[0, 1, :]]), bins[0],duration_of_window)
            
            #Set up output for correct batch * class matrix
            #output_for_loss = torch.transpose(output.clone()[0].clone(), 0, 1).clone().float()
            
            #Compute Loss
            #output = output_for_loss
            #print(output)
            
            #TODO REMOVE
            #print(output)
            #print("============")
            #m = nn.Softmax(dim=1)
            #output = m(output)
            #print(output)
            #TODO REMOVE
            
            #label_array =label_for_output.to(device)
            #label_array = label_array.long().to(device)
            
            #print(label_array, output)
            #loss = criterion(output, label_array)
            
            #Backprop + Check for learning
            #state_a = net_wrapper.model.state_dict().__str__()
            #loss.backward()
            #optimizer.step()
            #state_b = net_wrapper.model.state_dict().__str__()
            #if state_a == state_b:
            #    print("Network not updating.")


            # print statistics
            #running_loss += loss.item()
            #batch_loss += loss.item()
            #mini_batch_count += 1
            #batch_count += 1
            #print("=======================")
        
        #print(f'[{epoch + 1, mini_batch_count}] loss: {running_loss / mini_batch_count:.3f}')
        file_count += 1
        #if(file_count > 5):
        print("Processed ", file_count, " files")
        
    print("1: ", ones/total_count)
    print("0: ", zeros/total_count)
    print(f'[{epoch + 1, batch_count}] loss: {batch_loss / batch_count:.3f}')
    batch_loss = 0.0
    batch_count = 0
    break

print('Finished Training')
torch.save(net.state_dict(), "./test.h5")

Processed  1  files
Processed  2  files
Processed  3  files
Processed  4  files
Processed  5  files
Processed  6  files
Processed  7  files
Processed  8  files
Processed  9  files
Processed  10  files
Processed  11  files
Processed  12  files
Processed  13  files
Processed  14  files
Processed  15  files
Processed  16  files
Processed  17  files
Processed  18  files
Processed  19  files
Processed  20  files
Processed  21  files
Processed  22  files
Processed  23  files
Processed  24  files
Processed  25  files
Processed  26  files
Processed  27  files
Processed  28  files
Processed  29  files
Processed  30  files
Processed  31  files
Processed  32  files
Processed  33  files
Processed  34  files
Processed  35  files
Processed  36  files
Processed  37  files
Processed  38  files
Processed  39  files
Processed  40  files
Processed  41  files
Processed  42  files
Processed  43  files
Processed  44  files
Processed  45  files
Processed  46  files
Processed  47  files
Processed  48  files
P

Processed  379  files
Processed  380  files
Processed  381  files
Processed  382  files
Processed  383  files
Processed  384  files
Processed  385  files
Processed  386  files
Processed  387  files
Processed  388  files
Processed  389  files
Processed  390  files
Processed  391  files
Processed  392  files
Processed  393  files
Processed  394  files
Processed  395  files
Processed  396  files
Processed  397  files
Processed  398  files
Processed  399  files
Processed  400  files
Processed  401  files
Processed  402  files
Processed  403  files
Processed  404  files
Processed  405  files
Processed  406  files
Processed  407  files
Processed  408  files
Processed  409  files
Processed  410  files
Processed  411  files
Processed  412  files
Processed  413  files
Processed  414  files
Processed  415  files
Processed  416  files
Processed  417  files
Processed  418  files
Processed  419  files
Processed  420  files
Processed  421  files
Processed  422  files
Processed  423  files
Processed 

ZeroDivisionError: float division by zero

In [ ]:
net_wrapper.model.eval()
%reload_ext autoreload
correct = 0
total = 0
#Use Pyha with new weights to test performance
isolation_parameters = {
    "model" : "tweetynet",
     "tweety_output": True,
    "technique" : "steinberg",
     "threshold_type" : "median",
     "threshold_const" : 2.0,
     "threshold_min" : 0.0,
     "window_size" : 2.0,
     "chunk_size" : 5.0
}
automated_df = generate_automated_labels(path,isolation_parameters, weight_path="./test.h5");
display(automated_df)
manual_df = pd.read_csv("mixed_bird_manual.csv")
statistics_df = automated_labeling_statistics(automated_df,manual_df,stats_type = "general");
statistics_df
global_dataset_statistics(statistics_df)